# Automatic metrics 

Metrics, used for evaluating the quality of generated texts

- [x] ROUGE-1_recall
- [x] ROUGE-2_recall
- [x] ROUGE-L_recall
- [x] BLEU
- [x] METEOR
- [x] Perplexity 
- [x] BERTscore_precision
- [x] BERTscore_recall
- [x] BERTscore_f1
- [x] BLEURT

In [1]:
import torch
import typing as tp
import numpy as np

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Perplexity

$$Perplexity = \exp \bigl(- \frac{1}{N} \sum_{i=0}^{N} \log P(x_i | x_0, ..., x_{i-1})\bigr) =  \exp (\text{ouputs.loss})$$

$$\text{ouputs.loss} = - \frac{1}{N} \sum_{i=0}^{N} \log P(x_i | x_0, ..., x_{i-1})$$

$$ P(x_i | x_0, ..., x_{i-1}) = Softmax (\text{ouputs.logits})$$

$$Softmax (x_i) = \frac{e^{x_i}}{\sum_{j}e^{x_j}} $$

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "ai-forever/rugpt3large_based_on_gpt2"

ppl_tokenizer = AutoTokenizer.from_pretrained(model_name)
ppl_model = AutoModelForCausalLM.from_pretrained(model_name)
ppl_model.to(DEVICE)
ppl_model.eval()

def get_loss(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    text: str
):
    """
    Calculates loss: - mean of log p(x_i | x_0, ..., x_{i - 1})
    """
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss

    return loss

def calculate_perplexity(loss: torch.tensor):
    """
    Calculates perplexity: exp( - mean of log p(x_i | x_0, ..., x_{i - 1}))
    """
    perplexity = torch.exp(loss).item()
    return perplexity
    

In [3]:
assert calculate_perplexity(torch.tensor(0.)) == 1.

In [4]:
assert round(calculate_perplexity(torch.tensor(1.)), 5) == 2.71828

In [5]:
calculate_perplexity(get_loss(ppl_model, ppl_tokenizer, "Кот съел рыбу."))

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


202.2754669189453

## ROUGE 

<!-- $$\text{ROUGE-N} = \frac{\text{Common n-gramm count}}{\text{Count of all n-gramm in reference}}$$ -->


$$ 
ROUGE^N_{recall} = \frac{\textit{N-grams that appear in both R and C}}{\textit{N-grams in R}}
$$ 

$$ 
ROUGE^N_{precision} = \frac{\textit{N-grams that appear in both R and C}}{\textit{N-grams in C}}
$$ 

$$ 
ROUGE^N_{F_1} = \frac{2 \cdot ROUGE^N_{recall} \cdot ROUGE^N_{precision}}{ROUGE^N_{recall} + ROUGE^N_{precision}}
$$ 

$R$ - reference, 

$C$ - candidate.

In [6]:
import string
from collections import defaultdict
from nltk.stem import SnowballStemmer
from nltk.util import ngrams

stemmer = SnowballStemmer("russian")


def preprocess_text(text: str, stemmer: SnowballStemmer = stemmer):
    """
    Uses stemming to transform words into initial form
    """
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator).lower()
    
    words = text.split()
    stems = [stemmer.stem(word) for word in words]
    return stems


def rouge_n(cand_stems: tp.List[str], ref_stems: tp.List[str], n: int = 1):
    """
    Calculates ROUGE-N for given n. Uses stems - lists of words in initial form 
    """
    candidate_ngrams = list(ngrams(cand_stems, n))
    reference_ngrams = list(ngrams(ref_stems, n))
    
    ref_counts = defaultdict(int)
    for gram in reference_ngrams:
        ref_counts[gram] += 1
    
    overlap = 0
    cand_counts = defaultdict(int)
    for gram in candidate_ngrams:
        cand_counts[gram] += 1
        if cand_counts[gram] <= ref_counts[gram]:
            overlap += 1
    
    precision = overlap / len(candidate_ngrams) if candidate_ngrams else 0
    recall = overlap / len(reference_ngrams) if reference_ngrams else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'rouge-{}_precision'.format(n): precision,
        'rouge-{}_recall'.format(n): recall,
        'rouge-{}_f1'.format(n): f1,
    }


def lcs_rec(s1: tp.List[str], s2: tp.List[str], m: int, n: int, memo: tp.List[tp.List[int]]):
    """
    Calculates LCS (Longest Common Subsequence) using dynamic programming
    """
    if m == 0 or n == 0:
        return 0

    if memo[m][n] != -1:
        return memo[m][n]

    if s1[m - 1] == s2[n - 1]:
        memo[m][n] = 1 + lcs_rec(s1, s2, m - 1, n - 1, memo)
        return memo[m][n]

    memo[m][n] = max(lcs_rec(s1, s2, m, n - 1, memo),
                     lcs_rec(s1, s2, m - 1, n, memo))
    return memo[m][n]


def lcs(s1: tp.List[str], s2: tp.List[str]):
    """
    Calculates LCS (Longest Common Subsequence)
    """
    m = len(s1)
    n = len(s2)
    memo = [[-1 for _ in range(n + 1)] for _ in range(m + 1)]
    return lcs_rec(s1, s2, m, n, memo)


def rouge_l(cand_stems: tp.List[str], ref_stems: tp.List[str]):
    """
    Calculates ROUGE-L using LCS function for numerator calculation. 
    Uses stems - lists of words in initial form 
    """
    lcs_length = lcs(cand_stems, ref_stems)
    
    precision = lcs_length / len(cand_stems) if len(cand_stems) > 0 else 0
    recall = lcs_length / len(ref_stems) if len(ref_stems) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'rouge-l_lcs_length': lcs_length,
        'rouge-l_precision': precision,
        'rouge-l_recall': recall,
        'rouge-l_f1': f1
    }

candidate = "кот рыба ел"
reference = "кот рыба стол ел"

cand_stems = preprocess_text(candidate, stemmer)
ref_stems = preprocess_text(reference, stemmer)

rouge1 = rouge_n(cand_stems, ref_stems, n=1)
rouge2 = rouge_n(cand_stems, ref_stems, n=2)
rougel = rouge_l(cand_stems, ref_stems)
result = rouge1 | rouge2 | rougel

print(f"ROUGE-1 Precision: {result['rouge-1_precision']:.4f}")
print(f"ROUGE-1 Recall: {result['rouge-1_recall']:.4f}")
print(f"ROUGE-1 F1 Score: {result['rouge-1_f1']:.4f}")
print()

print(f"ROUGE-2 Precision: {result['rouge-2_precision']:.4f}")
print(f"ROUGE-2 Recall: {result['rouge-2_recall']:.4f}")
print(f"ROUGE-2 F1 Score: {result['rouge-2_f1']:.4f}")
print()

print(f"ROUGE-L LCS Length: {result['rouge-l_lcs_length']}")
print(f"ROUGE-L Precision: {result['rouge-l_precision']:.4f}")
print(f"ROUGE-L Recall: {result['rouge-l_recall']:.4f}")
print(f"ROUGE-L F1 Score: {result['rouge-l_f1']:.4f}")

ROUGE-1 Precision: 1.0000
ROUGE-1 Recall: 0.7500
ROUGE-1 F1 Score: 0.8571

ROUGE-2 Precision: 0.5000
ROUGE-2 Recall: 0.3333
ROUGE-2 F1 Score: 0.4000

ROUGE-L LCS Length: 3
ROUGE-L Precision: 1.0000
ROUGE-L Recall: 0.7500
ROUGE-L F1 Score: 0.8571


### LCS test

In [7]:
assert lcs([x for x in "ABC"], [x for x in "ACD"]) == 2
assert lcs([x for x in "AGGTAB"], [x for x in "GXTXAYB"]) == 4
assert lcs([x for x in "ABC"], [x for x in "CBA"]) == 1

In [8]:
assert lcs([x for x in "abcde"], [x for x in "ace"]) == 3
assert lcs([x for x in "abc"], [x for x in "abc"]) == 3
assert lcs([x for x in "abc"], [x for x in "def"]) == 0

### ROUGE test

In [9]:
candidate = "Собака бежит по полю"
reference = "Собака бежит по полю"

cand_stems = preprocess_text(candidate, stemmer)
ref_stems = preprocess_text(reference, stemmer)

rouge1 = rouge_n(cand_stems, ref_stems, n=1)
rouge2 = rouge_n(cand_stems, ref_stems, n=2)
rougel = rouge_l(cand_stems, ref_stems)
result = rouge1 | rouge2 | rougel

target_result = {
    'rouge-1_precision': 1.0,
    'rouge-1_recall': 1.0,
    'rouge-1_f1': 1.0,
    
    'rouge-2_precision': 1.0,
    'rouge-2_recall': 1.0,
    'rouge-2_f1': 1.0,
    
    'rouge-l_lcs_length': 4,
    'rouge-l_precision': 1.0,
    'rouge-l_recall': 1.0,
    'rouge-l_f1': 1.0
}

assert result == target_result

In [10]:
candidate = "Быстрый кот ест рыбу"
reference = "Рыжий кот ест рыбу"

cand_stems = preprocess_text(candidate, stemmer)
ref_stems = preprocess_text(reference, stemmer)

rouge1 = rouge_n(cand_stems, ref_stems, n=1)
rouge2 = rouge_n(cand_stems, ref_stems, n=2)
rougel = rouge_l(cand_stems, ref_stems)
result = rouge1 | rouge2 | rougel

target_result = {
    'rouge-1_precision': 0.75,
    'rouge-1_recall': 0.75,
    'rouge-1_f1': 0.75,
    
    'rouge-2_precision': 2 / 3,
    'rouge-2_recall': 2 / 3,
    'rouge-2_f1': 2 / 3,
    
    'rouge-l_lcs_length': 3,
    'rouge-l_precision': 0.75,
    'rouge-l_recall': 0.75,
    'rouge-l_f1': 0.75
}

assert result == target_result

In [11]:
candidate = "Рыбу ест кот"
reference = "Кот ест рыбу"

cand_stems = preprocess_text(candidate, stemmer)
ref_stems = preprocess_text(reference, stemmer)

rouge1 = rouge_n(cand_stems, ref_stems, n=1)
rouge2 = rouge_n(cand_stems, ref_stems, n=2)
rougel = rouge_l(cand_stems, ref_stems)
result = rouge1 | rouge2 | rougel

target_result = {
    'rouge-1_precision': 1,
    'rouge-1_recall': 1,
    'rouge-1_f1': 1,
    
    'rouge-2_precision': 0,
    'rouge-2_recall': 0,
    'rouge-2_f1': 0,
    
    'rouge-l_lcs_length': 1,
    'rouge-l_precision': 1 / 3,
    'rouge-l_recall': 1 / 3,
    'rouge-l_f1': 1 / 3
}

assert result == target_result

In [12]:
candidate = "Кот, кот, кот"
reference = "Кот ест рыбу"

cand_stems = preprocess_text(candidate, stemmer)
ref_stems = preprocess_text(reference, stemmer)

rouge1 = rouge_n(cand_stems, ref_stems, n=1)
rouge2 = rouge_n(cand_stems, ref_stems, n=2)
rougel = rouge_l(cand_stems, ref_stems)
result = rouge1 | rouge2 | rougel

target_result = {
    'rouge-1_precision': 1 / 3,
    'rouge-1_recall': 1 / 3,
    'rouge-1_f1': 1 / 3,
    
    'rouge-2_precision': 0,
    'rouge-2_recall': 0,
    'rouge-2_f1': 0,
    
    'rouge-l_lcs_length': 1,
    'rouge-l_precision': 1 / 3,
    'rouge-l_recall': 1 / 3,
    'rouge-l_f1': 1 / 3
}

assert result == target_result

In [13]:
candidate = ""
reference = "Пример? текста для: теста"

cand_stems = preprocess_text(candidate, stemmer)
ref_stems = preprocess_text(reference, stemmer)

rouge1 = rouge_n(cand_stems, ref_stems, n=1)
rouge2 = rouge_n(cand_stems, ref_stems, n=2)
rougel = rouge_l(cand_stems, ref_stems)
result = rouge1 | rouge2 | rougel

target_result = {
    'rouge-1_precision': 0,
    'rouge-1_recall': 0,
    'rouge-1_f1': 0,
    
    'rouge-2_precision': 0,
    'rouge-2_recall': 0,
    'rouge-2_f1': 0,
    
    'rouge-l_lcs_length': 0,
    'rouge-l_precision': 0,
    'rouge-l_recall': 0,
    'rouge-l_f1': 0
}

assert result == target_result

## BLEU

<!-- $$\text{BLEU} = \text{BP} \cdot \bigl( \sum_{n=1}^4 \omega_n \log p_n \bigr)$$

$$\text{BP} = \text{e} ^ {-max(0, \frac{r}{c} - 1)}$$

$$w_i = \frac{1}{4}$$

$c$ - the length of the stemmed candidate 

$r$ - effective reference length; sum of the best match lengths for each candidate sentence in the corpus (minimal length difference of reference and candidate texts)

$p_n$ - n-gramm recall:

$$p_n = \frac{ \sum_{\text{stem-ngram} \in C} \min \bigl( \text{count}_C (\text{stem-ngram}), \max_{r \in R} \text{count}_r (\text{stem-ngram}) \bigr)}{\sum_{\text{stem-ngram} \in C} \text{count}_C (\text{stem-ngram})}$$ -->

$$
BLEU = min \Biggl(1, exp(1 - \frac{len(reference)}{len(output)})\Biggr)
\llap{\underbrace{\phantom{min  \Biggl(1, exp(1 - \frac{len(reference)}{len(output)})\Biggr)}}_{\text{brevity penalty }}}
\left( \prod_{i = 1}^{4}precision_i\right)^{1/4}
\llap{\underbrace{\phantom{\left(\prod_{i = 1}^{4}precision_i)^{1/4}\right)}}_{\text{n-gram overlap}}},
$$

$len(reference)$ - count of unigramms in reference,

$len(output)$ - count of unigramms in candidate

In [14]:
import evaluate

bleu_metric = evaluate.load("bleu")

In [15]:
# check en result equals ru result

bleu_results_en = bleu_metric.compute(predictions=["cat rotate"], references=["cat sees rotate"])
bleu_results_ru = bleu_metric.compute(predictions=["кот поворот"], references=["кот видит поворот"])

assert bleu_results_ru == bleu_results_en
bleu_results_ru

{'bleu': 0.0,
 'precisions': [1.0, 0.0, 0.0, 0.0],
 'brevity_penalty': 0.6065306597126334,
 'length_ratio': 0.6666666666666666,
 'translation_length': 2,
 'reference_length': 3}

In [16]:
# check preprocess_text() removes punctuation and do stemming correctly

predictions = "Кот важный ел: - рыбу."
references = "Кот важно ел рыбу"

predictions = " ".join(preprocess_text(predictions, stemmer))
references = " ".join(preprocess_text(references, stemmer))

bleu_target = {
    'bleu': 1.0,
    'precisions': [1.0, 1.0, 1.0, 1.0],
    'brevity_penalty': 1.0,
    'length_ratio': 1.0,
    'translation_length': 4,
    'reference_length': 4
}
bleu_results = bleu_metric.compute(predictions=[predictions], references=[references])
assert bleu_results == bleu_target

bleu_results

{'bleu': 1.0,
 'precisions': [1.0, 1.0, 1.0, 1.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 4,
 'reference_length': 4}

Another way to do it

In [17]:
from nltk.translate.bleu_score import sentence_bleu

reference = [['this', 'movie', 'was', 'awesome']]
candidate = ['this', 'movie', 'was', 'awesome', 'too']
score = sentence_bleu(reference, candidate)
print(score)

0.668740304976422


In [18]:
score = sentence_bleu(["этот кот сидел на ковре".split()], "кот сидел на ковре".split())
print(score)

0.7788007830714049


## METEOR

$$Precision = \frac{m}{w_c}$$
$$Recall = \frac{m}{w_r} $$
$$F_{mean} = \frac{10 \cdot Precision \cdot Recall}{Recall + 10 \cdot Precision}$$

where $m$ - unigrams that appear in both Reference and Candidate,

$w_c$ - unigrams in Candidate,

$w_r$ - unigrams in Reference.

$$p = 0.5 \left( \frac{c}{u_m} \right)^3$$

where $u_m$ – summary count of unigrams in Candidate.

$$METEOR = F_{mean} (1 - p)$$


In [19]:
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /home/ezuryy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ezuryy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ezuryy/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:
# check en result equals ru result

results_en = meteor.compute(predictions=["text on english"], references=["another text on english"])
results_ru = meteor.compute(predictions=["текст на русском"], references=["другой текст на русском"])

assert results_ru == results_en

results_ru

{'meteor': np.float64(0.754985754985755)}

## BERTscore
 
$$R_{\text{BERT}} = \frac{1}{|x|} \sum_{x_i \in x} \max_{\hat{x}_j \in \hat{x}} x_i^\top \hat{x}_j$$

$$P_{\text{BERT}} = \frac{1}{|\hat{x}|} \sum_{\hat{x}_j \in \hat{x}} \max_{x_i \in x} x_i^\top \hat{x}_j$$

$$F_{\text{BERT}} = \frac{2 P_{\text{BERT}} \cdot R_{\text{BERT}}}{P_{\text{BERT}} + R_{\text{BERT}}}$$

$x$ - embedding of reference,

$\hat{x}$ - embedding of candidate.

In [21]:
from evaluate import load

bertscore_model = load("bertscore")

In [22]:
results = bertscore_model.compute(predictions=["текст на русском языке"], references=["текст на русском"], lang="ru")
results

{'precision': [0.9174482822418213],
 'recall': [0.9397120475769043],
 'f1': [0.9284467101097107],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.49.0)'}

In [23]:
def calculate_bertscore(
    model, 
    references: tp.List[str], 
    candidates: tp.List[str]
):
    """
    Calculates BERTscore for pairs reference-candidate. 
    Returns mean BERTscore values.
    """
    result_bertscore = model.compute(predictions=candidates, references=references, lang="ru")
    return {
        "BERTscore_precision": round(float(np.mean(result_bertscore["precision"])), 4),
        "BERTscore_recall": round(float(np.mean(result_bertscore["recall"])), 4),
        "BERTscore_f1": round(float(np.mean(result_bertscore["f1"])), 4),
    }

In [24]:
# equal texts
calculate_bertscore(bertscore_model, ["текст, на русском"], ["текст на русском"])

{'BERTscore_precision': 0.9351,
 'BERTscore_recall': 0.8923,
 'BERTscore_f1': 0.9132}

In [25]:
# almost equal texts
calculate_bertscore(bertscore_model, ["текст на русском"], ["другой текст на русском"])

{'BERTscore_precision': 0.9226,
 'BERTscore_recall': 0.948,
 'BERTscore_f1': 0.9351}

In [26]:
# different texts without common words
calculate_bertscore(bertscore_model, ["текст, на русском"], ["кот съел рыбку"])

{'BERTscore_precision': 0.5901,
 'BERTscore_recall': 0.5988,
 'BERTscore_f1': 0.5944}

In [27]:
# different texts with common words
calculate_bertscore(bertscore_model, ["кот не съел собаку"], ["кот съел рыбку"])

{'BERTscore_precision': 0.8413,
 'BERTscore_recall': 0.8261,
 'BERTscore_f1': 0.8336}

## BLEURT

In [28]:
from bleurt_pytorch import BleurtConfig, BleurtForSequenceClassification, BleurtTokenizer  
import transformers


bleurt_config = BleurtConfig.from_pretrained('lucadiliello/BLEURT-20-D12')  
bleurt_model = BleurtForSequenceClassification.from_pretrained('lucadiliello/BLEURT-20-D12')  
bleurt_tokenizer = BleurtTokenizer.from_pretrained('lucadiliello/BLEURT-20-D12')  

bleurt_model.to(DEVICE)
bleurt_model.eval() 

transformers.logging.set_verbosity_error()

def calculate_bleurt(
    model: BleurtForSequenceClassification, 
    tokenizer: BleurtTokenizer, 
    references: tp.List[str], 
    candidates: tp.List[str]
):
    """
    Calculates BLEURT for pairs reference-candidate. 
    Returns mean BLEURT value.
    """
    with torch.no_grad():  
        inputs = tokenizer(
            references, candidates, 
            max_length=512, 
            truncation=True, 
            padding='longest',
            return_tensors='pt').to(DEVICE)
        scores = model(**inputs).logits.flatten().tolist() 
    
    return {
        "BLEURT": round(float(np.mean(scores)), 4)
    }

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BleurtSPTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [29]:
calculate_bleurt(bleurt_model, bleurt_tokenizer, ["Кот съел рыбку"], ["Кот съел рыбину"])

{'BLEURT': 0.8455}

In [30]:
references = [
    "кошка лежит на ковре",
    "нейронные сети развивают Natural Language Processing"
]

candidates = [
    "котик лежит на коврике",
    "нейросети улучшают обработку естественного языка"
]

calculate_bleurt(bleurt_model, bleurt_tokenizer, references, candidates)

{'BLEURT': 0.5484}

# Apply metrics


In [31]:
import pandas as pd

df = pd.read_csv("inference_result/df_test.csv")
df.head()

,url,text,title,source,timestamp,cleaned_title,cleaned_text,len_cleaned_text,gen_llama3_8b,gen_llama3_8b_lora,gen_mistral_7b_lora,gen_mistral_7b,gen_qwen_7b,gen_qwen_7b_lora,gen_llama_8b,gen_yagpt_8b,gen_rugpt_13b,gen_tliteit_7b,gen_llama_8b_instruct
0,http://www.fontanka.ru/2013/10/16/017/,Пожар произошел минувшей ночью в Московском ра...,На стоянке на Звездной обгорело семь машин,taiga_fontanka,1381911180,На стоянке на Звездной обгорело семь машин,Пожар произошел минувшей ночью в Московском ра...,476,```\nВ результате пожара в ночном клубе «Мечта...,В ночь с 28-го апреля на 29-е в поселковом цен...,Семь автомобилей горит в субботу утром на стоя...,Воскресенья утром в 10 часов недалеко от город...,В ночь с 19 на 20 августа в районе улицы Степа...,Сегодня утром в районе станции метро «Звёздная...,В среду утром в городе Бауманск произошло пожа...,\n**Пожар уничтожил несколько автомобилей на с...,В ночь с 13-го на 14 января в Санкт-Петербурге...,\n\nВчера вечером на одной из самых оживленных...,Вчера в 22:00 по местному времени дежурным охр...
1,https://tass.ru/sport/3866330,"МОСКВА, 13 декабря. /ТАСС/. Олимпийская чемпио...",Олимпийская чемпионка Елена Веснина стала поче...,ods_tass,1481617927,Олимпийская чемпионка Елена Веснина стала поче...,Олимпийская чемпионка по теннису россиянка Еле...,851,```\nСегодня в городе состоялась церемония наг...,Санкт-Петербургский городской комитет спорта и...,Двукратная олимпийская чемпионка в парных соре...,\nElena Vesnina became an honorary citizen of ...,Сегодня в городе Сочи состоялась церемония наг...,Сегодня в городе Сочи прошло торжественное мер...,Сегодня в городе на берегу Черного моря состоя...,\nЕлена Веснина - выдающаяся российская теннис...,В субботу в Олимпийском парке состоялось торже...,"\n\nСегодня в Сочи состоялась церемония, на ко...",Елена Васильевна Веснина — российский теннисис...
2,http://rusplt.ru/region-news/barnaul/v-altaysk...,С начала ноября четыре жителя Алтайского края ...,В Алтайском крае под лед провалились четыре че...,buriy,1447147996,В Алтайском крае под лед провалились четыре че...,С начала ноября четыре жителя Алтайского края ...,744,Сегодня утром в селе Барабинка Усть-Коксинског...,Четырех человек удалось вытащить из воды в Бий...,"По информации ГУ МЧС по региону, вчера вечером...",Алматинская телевизионная компания «Казахстан»...,Сегодня в поселке Новоселово Алтайского края п...,Сегодня в Бийске произошла трагическая ситуаци...,Пять человек отправились на рыбалку в районе с...,\n**Трагедия на льду в Алтайском крае: четверо...,"Четверо мужчин утонули, провалившись в прорубь...",**Название: Четверо провалились под лед на Алт...,Сегодня утром в одной из деревень Вершинского ...
3,https://lenta.ru/news/2002/08/06/chisinau/,Президент Молдавии Владимир Воронин своим указ...,Президент Молдавии уволил главу своей службы о...,lenta,1028592000,Президент Молдавии уволил главу своей службы о...,Президент Молдавии Владимир Воронин своим указ...,697,> Игорь Додон покинул должность в связи с расс...,Начальник Службой государственной безопасности...,"Владимир Плантеа, который был назначен премьер...",\nMoldovan President Nicolae Timofti has sacke...,В пятницу президент Республики Молдова Дорин К...,Министерство внутренних дел Республики Молдова...,Председатель парламента Молдовы Игорь Корман з...,\nВ молдавской столице Кишинёве сегодня произо...,В четверг президент Молдавии Владимир Воронин ...,"\n\nВчера президент Молдавии, Игорь Додон, при...",В четверг президент Молдовы Игорь Додон заявил...
4,https://lenta.ru/news/2017/08/21/annabel/,Посетительнице кинотеатра в Бразилии стало пло...,Бразильянке вызвали скорую после просмотра хор...,lenta,1503273600,Бразильянке вызвали скорую после просмотра хор...,Посетительнице кинотеатра в Бразилии стало пло...,655,```\nСкорая помощь была вызвана в бразильском ...,"Согласно информации сайта Ananova, 24-летняя б...",В Бразилии медики получили званок из кинотеатр...,"В Бразилии 20-летняя девушка попросила скорую,...",В Бразилии произ

In [ ]:
df.columns

Index(['url', 'text', 'title', 'source', 'timestamp', 'cleaned_title',
       'cleaned_text', 'len_cleaned_text', 'gen_llama3_8b',
       'gen_llama3_8b_lora', 'gen_mistral_7b_lora', 'gen_mistral_7b',
       'gen_qwen_7b', 'gen_qwen_7b_lora', 'gen_llama_8b', 'gen_yagpt_8b',
       'gen_rugpt_13b', 'gen_tliteit_7b', 'gen_llama_8b_instruct'],
      dtype='object')

In [ ]:
from tqdm import tqdm as tqdm_a

metric = {}

for col in tqdm_a(['gen_llama3_8b',
       'gen_llama3_8b_lora', 'gen_mistral_7b_lora', 'gen_mistral_7b',
       'gen_qwen_7b', 'gen_qwen_7b_lora', 'gen_llama_8b', 'gen_yagpt_8b',
       'gen_rugpt_13b', 'gen_tliteit_7b', 'gen_llama_8b_instruct']):
    
    sub_df = df.sample(n=100)
    candidates, references = sub_df[col].tolist(), sub_df['cleaned_text'].tolist()

    MAX_LENGTH = 2048

    metrics_list = []
    for candidate, reference in zip(candidates, references):
        # We use stemms for metrics without transformers and LLM
        candidate_stems = " ".join(preprocess_text(candidate, stemmer))
        reference_stems = " ".join(preprocess_text(reference, stemmer))
        
        # ROUGE
        rouge1 = rouge_n(candidate_stems, reference_stems, n=1)
        rouge2 = rouge_n(candidate_stems, reference_stems, n=2)
        rougel = rouge_l(candidate_stems[:MAX_LENGTH], reference_stems[:MAX_LENGTH]) 
        
        result = {
            'ROUGE-1 recall': rouge1['rouge-1_recall'],
            'ROUGE-1 precision': rouge1['rouge-1_precision'],
            'ROUGE-1 f1': rouge1['rouge-1_f1'],
        }
        result |= {
            'ROUGE-2 recall': rouge2['rouge-2_recall'],
            'ROUGE-2 precision': rouge2['rouge-2_precision'],
            'ROUGE-2 f1': rouge2['rouge-2_f1'],
        }
        result |= {
            'ROUGE-L recall': rougel['rouge-l_recall'],
            'ROUGE-L precision': rougel['rouge-l_precision'],
            'ROUGE-L f1': rougel['rouge-l_f1']
        }     

        # BLEU
        bleu_results = bleu_metric.compute(predictions=[candidate_stems], references=[reference_stems])
        result |= {'BLEU': bleu_results['bleu']}

        # METEOR
        meteor_results = meteor.compute(predictions=[candidate_stems], references=[reference_stems])
        result |= {'METEOR': float(meteor_results['meteor'])}

        # Perplexity
        perplexity = calculate_perplexity(get_loss(ppl_model, ppl_tokenizer, candidate))
        result |= {"Perplexity": perplexity}
        
        metrics_list.append(result)

    bertscore_metric = calculate_bertscore(bertscore_model, references, candidates)
    bleurt_metric = calculate_bleurt(bleurt_model, bleurt_tokenizer, references, candidates)
    
    metrics_dict = {k: round(float(np.mean([d[k] for d in metrics_list])), 4) for k in metrics_list[0]} 
    metric[col] = metrics_dict | bertscore_metric | bleurt_metric
    

 36%|████████████████████████████████████████████████████████████████                                                                                                                | 4/11 [00:53<01:37, 13.96s/it]

In [34]:
df_metric = pd.DataFrame(metric)
df_metric

,gen_llama3_8b,gen_llama3_8b_lora,gen_mistral_7b_lora,gen_mistral_7b,gen_qwen_7b,gen_qwen_7b_lora,gen_llama_8b,gen_yagpt_8b,gen_rugpt_13b,gen_tliteit_7b,gen_llama_8b_instruct
ROUGE-1 recall,0.6456,0.6418,0.5072,0.7556,0.9567,0.9614,0.8589,0.9364,0.5526,0.9712,0.9776
ROUGE-1 precision,0.5325,0.9090,0.9397,0.4562,0.5266,0.4807,0.6079,0.5664,0.9004,0.4674,0.3735
ROUGE-1 f1,0.5133,0.7209,0.6204,0.5525,0.6560,0.6261,0.6457,0.6732,0.6357,0.6206,0.5321
ROUGE-2 recall,0.4704,0.4641,0.3716,0.5537,0.7830,0.7920,0.6741,0.7533,0.4097,0.8145,0.8395
ROUGE-2 precision,0.3684,0.6703,0.7100,0.3303,0.4210,0.3899,0.4610,0.4427,0.6869,0.3888,0.3183
ROUGE-2 f1,0.3658,0.5250,0.4598,0.4025,0.5286,0.5107,0.4944,0.5317,0.4744,0.5175,0.4543
ROUGE-L recall,0.3716,0.3486,0.2936,0.4384,0.5696,0.5783,0.4966,0.5608,0.3163,0.6037,0.6364
ROUGE-L precision,0.3174,0.5089,0.5785,0.2586,0.2979,0.2790,0.3432,0.3196,0.5527,0.2819,0.2363
ROUGE-L f1,0.2925,0.3952,0.3649,0.3156,0.3766,0.3674,0.3608,0.3864,0.3711,0.3775,0.3388
BLEU,0.0010,0.0075,0.0100,0.0018,0.0052,0.0060,0.0033,0.0085,0.0081,0.0103,0.0018


In [ ]:
df_metric.to_excel("inference_result/auto_metric_3.xlsx")